In [40]:
import pandas as pd
import string
import re

from tqdm import tqdm
from tqdm.auto import tqdm

from translate import Translator
from langdetect import detect

from nltk.stem.porter import *
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

# Laad de data in

In [43]:
df = pd.read_csv("../data/uploads.csv").drop("Unnamed: 0", axis=1)

In [44]:
df.tail()

,channel,video_id,title,description,conspiracy,transcript
65679,UCgRvm1yLFoaQKhmaTqXk9SA,tt6yWP2VWOU,"New Zealand's Push for Gun Reform, Explained |...",New Zealand is making some big changes. Their ...,False,New Zealand is making some big changes. [ARDEN...
65680,UCgRvm1yLFoaQKhmaTqXk9SA,t7pmHD8rqOA,The Rise and Fall of Myanmar’s Aung San Suu Ky...,A non-violent freedom fighter? A war crimes ap...,False,A non-violent freedom fighter? War crimes apo...
65681,UCgRvm1yLFoaQKhmaTqXk9SA,K7pzjFOe7X0,Spain's Plan to Quit the Coal Industry Explain...,Climate change is happening. And one industry ...,False,Climate change is happening. And one industry ...
65682,UCgRvm1yLFoaQKhmaTqXk9SA,KpYxFouaTvk,How Costa Rica Avoided Cold War Violence | Now...,Costa Rica has sometimes been called “the Swit...,False,Costa Rica has sometimes been called “the Swi...
65683,UCgRvm1yLFoaQKhmaTqXk9SA,BiulFh5uAyM,Trump’s Historic 'National Emergency’ Explaine...,President Donald Trump has declared a national...,False,What you’re about to see… is a historic moment...


# Data opschonen

In [45]:
df["conspiracy"].value_counts()

False    43528
True     22156
Name: conspiracy, dtype: int64

In [46]:
# Balanceer de classes
nc = df[df["conspiracy"] == False].sample(22156)
c = df[df["conspiracy"] == True]

equal = pd.concat([nc, c]).reset_index().drop("index", axis=1)
equal.head()

,channel,video_id,title,description,conspiracy,transcript
0,UCH9gafn41oPg7s6bQwiMwTg,hgOdDCd6N10,Islam Beats Feminism,"Your rights end where my stone begins, motherf...",False,[Music] what do you get when you combine a gan...
1,UCHr26UnUE_FHYtAXc_ddfgg,um-kBMGmW5k,24 Hours at Burning Man 2019 ... Micro Edition,A little taste of Burning Man 2019 .... expect...,False,NaN
2,UCsu6NM_ARGWzDwL1C9xEhqA,9Z2CMOdLoB8,Police Reform,NaN,False,in a time of turmoil and injustice the world c...
3,UCTjxmzChimJa3X_rAgLAnxg,GNJny3dtmoQ,"""Careers""",Jobs not careers!\n\nSupport my video-making! ...,False,hi everyone so the other day i got a notificat...
4,UCaoSDiNkFmGQfvhvZMWxOHw,lOoCe0OZgvc,How Was Socrates Able to Predict the Civil Unr...,"Comments on current events with economist, Mar...",False,how did socrates forecast the civil unrest tha...


In [47]:
# Vul lege cellen en lowercase
equal["transcript"] = equal["transcript"].fillna("")
equal["title"] = equal["title"].fillna("")
equal["description"] = equal["description"].fillna("")

equal["transcript"] = equal["transcript"].str.lower()
equal["title"] = equal["title"].str.lower()
equal["description"] = equal["description"].str.lower()

In [48]:
# Voeg titel, beschrijving en transcript samen tot één kolom
equal["full_text"] = equal["title"] + " " + equal["description"] + " " + equal["transcript"]

equal = equal.drop(["title", "description", "transcript"], axis = 1)

In [49]:
def translate_text(text):
    try:
        # Detect language
        lang = detect(text)
        # If the text is not in English, translate it, otherwise just return
        if lang != "en":
            translator = Translator(to_lang="en", from_lang=lang)
            return translator.translate(text)
        else:
            return text
    except: # Return empty texts as-is
        return text

In [50]:
tqdm.pandas()
# Vertaal alle text naar het Engels
equal['full_text'] = equal['full_text'].progress_apply(lambda txt: translate_text(txt))

  0%|          | 0/44312 [00:00<?, ?it/s]

In [51]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [52]:
def stem_text(text):
    stripped = [word.strip(string.punctuation) for word in word_tokenize(text) if word not in stop_words]
    return " ".join([stemmer.stem(word) for word in stripped if word])

In [53]:
# Stem transcripts en verwijder stopwords
equal['full_text'] = equal['full_text'].progress_apply(lambda text: stem_text(text))

  0%|          | 0/44312 [00:00<?, ?it/s]

In [54]:
equal["full_text"].fillna("", inplace=True)

In [55]:
equal.to_csv("training_videos.csv")